In [1]:
import sys
import os
import findspark
import os


In [2]:
USER = os.environ['USER']
print(USER)

r0t00xk


In [3]:
HADOOP_PATH = "/opt/spark-2.3.0-bin-hadoop2.7"
HADOOP_JARS_PATH = HADOOP_PATH + "/jars/"

In [4]:
findspark.init(HADOOP_PATH)
#import pyspark
import pyspark
from pyspark.sql import SparkSession

In [5]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType
import datetime
from datetime import datetime
from pyspark.sql.functions import col,from_json,date_format
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import time

In [6]:
pyspark_submit_args= "--master yarn \
--deploy-mode client \
--executor-memory 6G \
--driver-memory 5G \
--queue queue2 \
--executor-cores 4 \
--num-executors 20 \
--conf spark.yarn.dist.archives=gs://p13n-storage2/"+USER+"/envs/dev27.zip#ecop27 \
--archives gs://p13n-storage2/r0t00xk/envs/dev27.zip#ecop27 \
--jars "+HADOOP_JARS_PATH+"/datanucleus-api-jdo-3.2.6.jar,"+HADOOP_JARS_PATH+"/datanucleus-rdbms-3.2.9.jar,"+HADOOP_JARS_PATH+"/datanucleus-core-3.2.10.jar,/u/users/r0t00xk/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.11-2.3.0.jar,/u/users/r0t00xk/.ivy2/jars/org.apache.kafka_kafka-clients-0.10.1.0.jar \
pyspark-shell"

In [7]:
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
os.environ["PYSPARK_PYTHON"] = './ecop27/dev27/bin/python'
os.environ["PYSPARK_DRIVER_PYTHON"] = './ecop27/dev27/bin/python'

In [8]:
schema = StructType([
  StructField("pageType",StringType(),True),
  StructField("pageContext",StringType(),True),
  StructField("deviceType",StringType(),True),
  StructField("details",ArrayType(StringType()),True),
  StructField("timestamp",StringType(),True),
  StructField("reqId",StringType(),True),
  StructField("betaCoeff",StringType(),True),
  ]
)

In [9]:
spark = SparkSession.builder.getOrCreate()

In [10]:
def paritiondate():
  return datetime.today().strftime('%Y%m%d-%H%M')
udfDateHRMM= udf(paritiondate, StringType())

In [11]:
# Read Kafka Messages as spark structured stream
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers","kafka-1054180686-1-1268601033.wus.kafka-v2-sp-ad-server-prod.ms-df-messaging.prod-az-westus-23.prod.us.walmart.net:9092").option("subscribe", "sp_p13n_request_logs").option("multiLine", "true").option("startingOffsets", "latest").option("failOnDataLoss","false").load().selectExpr("CAST(value AS STRING) as kafkamsgval").withColumn("hour",udfDateHRMM())

In [12]:
#Kafka messages have their own schema, define own schema for storage and read kafka json as own defined schema
dv = df.withColumn("rootData",from_json(col("kafkamsgval"),schema)) \
                   .select("rootData")
dv.printSchema()

root
 |-- rootData: struct (nullable = true)
 |    |-- pageType: string (nullable = true)
 |    |-- pageContext: string (nullable = true)
 |    |-- deviceType: string (nullable = true)
 |    |-- details: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- timestamp: string (nullable = true)
 |    |-- reqId: string (nullable = true)
 |    |-- betaCoeff: string (nullable = true)



In [13]:
#rootData is one main json blob, read that into individual columns
mainDF= dv.select("rootData.pageType", "rootData.pageContext", "rootData.deviceType", "rootData.timestamp", "rootData.reqId","rootData.betaCoeff")
# from pyspark.sql.functions import *
# mainDF.withColumn('hour', hour(from_unixtime('timestamp'))).show()
mainDF.printSchema()

root
 |-- pageType: string (nullable = true)
 |-- pageContext: string (nullable = true)
 |-- deviceType: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- reqId: string (nullable = true)
 |-- betaCoeff: string (nullable = true)



In [14]:
#spark sql init, create table adlogs from dataframe
mainDF.createTempView("adlogs")
#you can execute sql like queries like below
df2 = spark.sql("select * from adlogs")

In [15]:
consoleoutput = df.writeStream.trigger(processingTime='1 seconds').partitionBy('hour').format("parquet").option("path","gs://p13n-storage2/r0t00xk/dump").outputMode("append").option("checkpointLocation", "checkpoint").option("truncate", "false").start().awaitTermination()


StreamingQueryException: u'Job aborted.\n=== Streaming Query ===\nIdentifier: [id = 225ef25a-312a-4b46-97d3-ef201458eeb8, runId = f9b39040-82f5-4466-b0b1-71e32481cc8a]\nCurrent Committed Offsets: {KafkaSource[Subscribe[sp_p13n_request_logs]]: {"sp_p13n_request_logs":{"8":108878023,"11":108887676,"2":108885204,"5":108893843,"14":108897080,"13":108888518,"4":108887228,"7":108900061,"1":108889302,"10":108878878,"9":108898059,"3":108869338,"12":111953736,"6":108901414,"0":108901352}}}\nCurrent Available Offsets: {KafkaSource[Subscribe[sp_p13n_request_logs]]: {"sp_p13n_request_logs":{"8":111541505,"11":111550786,"2":111547509,"5":111555540,"14":111559869,"13":111551683,"4":111550388,"7":111563848,"1":111553036,"10":111541259,"9":111560083,"3":111532753,"12":113459953,"6":111563658,"0":111562650}}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nProject [kafkamsgval#21, paritiondate() AS hour#23]\n+- Project [cast(value#8 as string) AS kafkamsgval#21]\n   +- StreamingExecu

In [ ]:
# spark.streams.awaitAnyTermination()

In [ ]:
# spark.stop()